In [1]:
import os
import re

In [33]:
class DatamartOperator:
    TARGET_REGEX = r'.*?\"(.*)\"'
    DROP_TARGET = """
    DROP TABLE datarmat.{0};
    """
    def __init__(self, dag, folder):
        self.dag = dag
        self.folder = folder

    def list_files_in_folder(self):
        return [ x for x in os.listdir(self.folder) if ".sql" in x]

    def get_target_table(self, content):
        target_table = re.findall(self.TARGET_REGEX, content)[0]
        return target_table


    def read_sql_file(self, file):
        with open(os.path.join(self.folder, file), "r") as f:
            content = f.read()
        self.current_content = content
        return content

    def build_task_query(self, content):
        target_table = self.get_target_table(content)
        return self.DROP_TARGET.format(target_table) + content

    # def build_task(self, file):
    #     content = self.read_sql_file(file)
    #     task_id = "task__" + file.split(".")[0]
    #     task_obj = PosgresOperator(
    #         task_id = task_id,
    #         sql=content, 
    #         dag = self.dag
    #     )
        
    # def build_tasks(self):
    #     for f in self.folder():
    #         self.build_task(f)




    


In [34]:
factory = DatamartOperator(dag="dag", folder='..')
content = factory.read_sql_file(factory.list_files_in_folder()[0])
print(factory.build_task_query(content))


    DROP TABLE datarmat.mart_daily_performance;
    CREATE TABLE datamart."mart_daily_performance" AS
    select
    purchase_date
    , customer_state
    , order_status
    , count(order_id) as count_orders
    , sum(payment_total_sum) as payment_sum
    , sum(total_freight) as freight_sum
    , sum(total_price) as total_costs
    from public."orders"
    group by 1,2,3;


In [17]:
re.findall(r'.*?\"(.*)\"',content)[0]

'mart_daily_performance'